# Quora Question Pairs

- [Quora Question Pairs challengeQuora Question Pairs challenge](https://www.kaggle.com/c/quora-question-pairs).
- 쿼라(Quora)는 사용자 커뮤니티이며, 질문자(Seeker)는 궁금한 사항을 올리고, 답변자(Writer)는 해당 사항에 답변하는 형식의 질의 응답형 웹사이트입니다.쿼라(Quora)는 사용자 커뮤니티이며, 질문자(Seeker)는 궁금한 사항을 올리고, 답변자(Writer)는 해당 사항에 답변하는 형식의 질의 응답형 웹사이트입니다.

## 0. Library 설치

In [ ]:
# kaggle에 있는 데이터를 불러옵니다.
# os library는 운영체제(OS)에서 제공되는 여러 기능을 파이썬에서 수행할 수 있게 해줍니다.
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Default 라이브러리
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from subprocess import check_output
import math

# visualization 라이브러리
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

# system 라이브러리
import sys
import csv
import gc # garbage collector
import sqlite3
import os
from os import path


# 정규표현식 관련 
import re

# 시간 데이터 처리를 위한 라이브러리
import time

# Progress bar 관련 라이브러리
from tqdm import tqdm

# NLP 관련 라이브러리
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Web Crawling 관련
from bs4 import BeautifulSoup

# 두 문자열 간의 유사성을 측정하는데 사용되는 
from fuzzywuzzy import fuzz

# scikit-learn 라이브러리
from sklearn import model_selection
from sklearn.calibration import CalibratedClassifierCV
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix, normalized_mutual_info_score
from sklearn.metrics import precision_recall_curve, auc, roc_curve
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, RandomizedSearchCV 
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from sklearn.svm import SVC


# 말구름(wordcloud) 그려주는 라이브러리
from wordcloud import WordCloud, STOPWORDS

# PIL: Python Image Library, 
# 설치시 pil이 아닌 image를 사용해야 함. pip install image
from PIL import Image

%matplotlib inline

# Extract Word2Vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools

# 파이썬의 자연어 처리를 라이브러리
import spacy
from scipy.sparse import hstack

import xgboost as xgb
from joblib import dump, load
from collections import Counter, defaultdict
from sqlalchemy import create_engine # Database connection
from mlxtend.classifier import StackingClassifier

In [ ]:
# %load "../input/d/elemento/quora-question-pairs/distance/distance/_lcsubstrings.py"
from array import array

def lcsubstrings(seq1, seq2, positions=False):    
	print("== START lcsubstrings ==")
	'''
	positions=False인 경우
	문자열 `seq1`과 `seq2`에서 가장 길게 겹치는 문자열을 찾으면, 문자열 자체를 반환함.
		>>> lcsubstrings("sedentar", "dentist", positions=True)
		>>> (4, [(2, 0)])

	positions=True인 경우
	문자열 `seq1`과 `seq2`에서 가장 길게 겹치는 문자열을 찾으면, 그 길이와 위치를 반환함.
		>>> lcsubstrings("sedentar", "dentist")
		>>> {'dent'}
	'''
	L1, L2 = len(seq1), len(seq2)
	ms = []
	mlen = last = 0
	if L1 < L2:
		seq1, seq2 = seq2, seq1
		L1, L2 = L2, L1
	
	column = array('L', range(L2))
	
	for i in range(L1):
		for j in range(L2):
			old = column[j]
			if seq1[i] == seq2[j]:
				if i == 0 or j == 0:
					column[j] = 1
				else:
					column[j] = last + 1
				if column[j] > mlen:
					mlen = column[j]
					ms = [(i, j)]
				elif column[j] == mlen:
					ms.append((i, j))
			else:
				column[j] = 0
			last = old
	
	if positions:
		return (mlen, tuple((i - mlen + 1, j - mlen + 1) for i, j in ms if ms))
	return set(seq1[i - mlen + 1:i + 1] for i, _ in ms if ms)

# 1. 문제점
## 1.1 프로젝트 설명  
- 문제는 유사한 질문들이 많이 올라오기 때문에 질문자(Seeker)는 좋은 답변을 찾기가 힘들고, 답변자(writer)는 사실상 동일한 답변에 대해 다시 답변을 해야 하기 때문에 부담이 됩니다. 문제는 유사한 질문들이 많이 올라오기 때문에 질문자(Seeker)는 좋은 답변을 찾기가 힘들고, 답변자(writer)는 사실상 동일한 답변에 대해 다시 답변을 해야 하기 때문에 부담이 됩니다. 
- 현재 Quora는 Random Forest 모델을 이용하여 중복되는 질문이 동일한 질문인지 식별하고 있습니다. 우리는 더 나은 사용자 경험을 제공하기 위해 질문 쌍들이 중복되는지 해결하여야 합니다.현재 Quora는 Random Forest 모델을 이용하여 중복되는 질문이 동일한 질문인지 식별하고 있습니다. 우리는 더 나은 사용자 경험을 제공하기 위해 질문 쌍들이 중복되는지 해결하여야 합니다.- 문제는 유사한 질문들이 많이 올라오기 때문에 질문자(Seeker)는 좋은 답변을 찾기가 힘들고, 답변자(writer)는 사실상 동일한 답변에 대해 다시 답변을 해야 하기 때문에 부담이 됩니다. 문제는 유사한 질문들이 많이 올라오기 때문에 질문자(Seeker)는 좋은 답변을 찾기가 힘들고, 답변자(writer)는 사실상 동일한 답변에 대해 다시 답변을 해야 하기 때문에 부담이 됩니다.
- 정답(ground truth)와 예측값(predicted valuue)간의 log loss 값으로 평가를 진행합니다.정답(ground truth)와 예측값(predicted valuue)간의 log loss 값으로 평가를 진행합니다.### 1. 프로젝트 설명  
- 문제는 유사한 질문들이 많이 올라오기 때문에 질문자(Seeker)는 좋은 답변을 찾기가 힘들고, 답변자(writer)는 사실상 동일한 답변에 대해 다시 답변을 해야 하기 때문에 부담이 됩니다. 문제는 유사한 질문들이 많이 올라오기 때문에 질문자(Seeker)는 좋은 답변을 찾기가 힘들고, 답변자(writer)는 사실상 동일한 답변에 대해 다시 답변을 해야 하기 때문에 부담이 됩니다. 
- 현재 Quora는 Random Forest 모델을 이용하여 중복되는 질문이 동일한 질문인지 식별하고 있습니다. 우리는 더 나은 사용자 경험을 제공하기 위해 질문 쌍들이 중복되는지 해결하여야 합니다.현재 Quora는 Random Forest 모델을 이용하여 중복되는 질문이 동일한 질문인지 식별하고 있습니다. 우리는 더 나은 사용자 경험을 제공하기 위해 질문 쌍들이 중복되는지 해결하여야 합니다.- 문제는 유사한 질문들이 많이 올라오기 때문에 질문자(Seeker)는 좋은 답변을 찾기가 힘들고, 답변자(writer)는 사실상 동일한 답변에 대해 다시 답변을 해야 하기 때문에 부담이 됩니다. 문제는 유사한 질문들이 많이 올라오기 때문에 질문자(Seeker)는 좋은 답변을 찾기가 힘들고, 답변자(writer)는 사실상 동일한 답변에 대해 다시 답변을 해야 하기 때문에 부담이 됩니다.
  
## 1.2 평가 방법
- 정답(ground truth)와 예측값(predicted valuue)간의 log loss 값으로 평가를 진행합니다.정답(ground truth)와 예측값(predicted valuue)간의 log loss 값으로 평가를 진행합니다.

## 1.3 관련 링크
- Source : https://www.kaggle.com/c/quora-question-pairs
- Discussions : https://www.kaggle.com/anokas/data-analysis-xgboost-starter-0-35460-lb/comments
- Kaggle Winning Solution and other approaches: https://www.dropbox.com/sh/93968nfnrzh8bp5/AACZdtsApc1QSTQc7X0H3QZ5a?dl=0
- Blog 1 : https://engineering.quora.com/Semantic-Question-Matching-with-Deep-Learning
- Blog 2 : https://towardsdatascience.com/identifying-duplicate-questions-on-quora-top-12-on-kaggle-4c1cf93f1c30

# 2. Machine Learning 관련 내용
## 2.1 파일 설명
<pre>
- training을 위한 파일
- 파일 크기: 60MB 
- 열(column) 정보 : id, qid1, qid2, question1, question2, is_duplicate
- 행의 수: 404,290
</pre>
## 2.2 train.csv의 열(col) 정보
<pre>
- question1 및 question2 : 질문을 포함하는 문자열 1, 2
- qid1 및 qid2 : question1 및 question2의 구분자
- id : 질문 쌍 구분자
- is_duplicate : 중복 여부에 대한 labeling
</pre>

# 3. EDA(Exploratory Data Analysis)
## 3.1 data 가져오기

In [ ]:
df = pd.read_csv("../input/quora-question-pairs/train.csv.zip")
print(df.info())
df.head()

### 3.2.1 데이터 분포
- 유사, 비유사 쌍의 수와 비율

In [ ]:
total_cnt = len(df)
similar_cnt = df.groupby("is_duplicate")['id'].count()[0]
not_similar_cnt = df.groupby("is_duplicate")['id'].count()[1]

print("전체: %d \n비유사(is_duplicate=0): %d \n유사(is_duplicate=1): %d" % (total_cnt, similar_cnt, not_similar_cnt))
print('유사 쌍의 비율 (is_duplicate = 1): %2.2f%%' % (similar_cnt/total_cnt*100))
print('비유사 쌍의 비율 (is_duplicate = 0): %2.2f%%' % (not_similar_cnt/total_cnt*100))

### 3.2.2 질문의 중복 확인

In [ ]:
# 질문의 중복을 확인하기 위해 qid는 1번이상 사용되므로, 이를 이용하여 질문의 중복정도를 체크한다.
qids = pd.Series(df['qid1'].tolist() + df['qid2'].tolist())

# 전체 질문의 수 
unique_qs = len(np.unique(qids))

# 두번이상 나온 질문
qs_morethan_onetime = np.sum(qids.value_counts() > 1)
qs_morethan_onetime_rate = qs_morethan_onetime/unique_qs*100

print('전체 질문의 수: %d' % unique_qs)
print('두 번 이상 반복된 질문의 수: %d, 비율: %2.2f%%' % (qs_morethan_onetime, qs_morethan_onetime_rate))
print('가장 많이 반복되는 질문의 반복 횟수: %d' % max(qids.value_counts())) 

q_vals = qids.value_counts()
q_vals = q_vals.values

### 3.2.3 중복 확인

In [ ]:
pair_duplicates = df[['qid1','qid2','is_duplicate']].groupby(['qid1','qid2']).count().reset_index()
print ("Number of duplicate questions", (pair_duplicates).shape[0] - df.shape[0])

### 3.2.4 각 질문의 발생 수

In [ ]:
print ('가장 많이 반복된 질문의 빈도: %d' % max(qids.value_counts())) 

plt.figure(figsize=(20, 10))
plt.hist(qids.value_counts(), bins=160)
plt.yscale('log', nonpositive='clip')
plt.title('Log-Histogram of question appearance counts')
plt.xlabel("Question occurences counts")
plt.ylabel('Questions counts')

### 3.2.5 NULL값 확인

In [ ]:
# null겂을 가진 행이 있는지 확인
nan_rows = df[df.isnull().any(1)]
print (nan_rows)

In [ ]:
# question2에서 2개의 행에 Nan이 있으므로 na를 채우고, 확인
# Filling the null values with ' '
df = df.fillna('')
nan_rows = df[df.isnull().any(1)]
print (nan_rows)

## 3.3 기본적인 특성 추출
 - ____freq_qid1____ = qid1의 빈도
 - ____freq_qid2____ = qid2의 빈도
 - ____q1len____ = q1의 길이
 - ____q2len____ = q2의 길이
 - ____q1_n_words____ = question 1의 단어 수
 - ____q2_n_words____ = question 2의 단어수
 - ____word_Common____ = question1과 question2에서 동일하게 나타나는 단어 수
 - ____word_Total____ = Question1의 전체 단어수와 Question2의 전체 단어 수의 합
 - ____word_share____ = word_common / word_Total
 - ____freq_q1+freq_q2____ = qid1과 qid2의 빈도 합계
 - ____freq_q1-freq_q2____ = qid1과 qid2의 빈도의 차이의 절대값

In [ ]:
if os.path.isfile('../input/d/elemento/quora-question-pairs/df_fe_without_preprocessing_train.csv'):
    print("os.path.isfile: TRUE")
    df = pd.read_csv("../input/d/elemento/quora-question-pairs/df_fe_without_preprocessing_train.csv", encoding='latin-1')
else:
    print("os.path.isfile:  FALSE")
    df['freq_qid1'] = df.groupby('qid1')['qid1'].transform('count') 
    df['freq_qid2'] = df.groupby('qid2')['qid2'].transform('count')
    df['q1len'] = df['question1'].str.len() 
    df['q2len'] = df['question2'].str.len()
    df['q1_n_words'] = df['question1'].apply(lambda row: len(row.split(" ")))
    df['q2_n_words'] = df['question2'].apply(lambda row: len(row.split(" ")))

    def normalized_word_Common(row):
        w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
        w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
        return 1.0 * len(w1 & w2)
    df['word_Common'] = df.apply(normalized_word_Common, axis=1)

    def normalized_word_Total(row):
        w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
        w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
        return 1.0 * (len(w1) + len(w2))
    df['word_Total'] = df.apply(normalized_word_Total, axis=1)

    def normalized_word_share(row):
        w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
        w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
        return 1.0 * len(w1 & w2)/(len(w1) + len(w2))
    df['word_share'] = df.apply(normalized_word_share, axis=1)

    df['freq_q1+q2'] = df['freq_qid1']+df['freq_qid2']
    df['freq_q1-q2'] = abs(df['freq_qid1']-df['freq_qid2'])

    df.to_csv("df_fe_without_preprocessing_train.csv", index=False)

df.head()

In [ ]:
print(df['question1'][0])
print(df['question2'][0])

### 3.3.1 추출된 특성의 분석

In [ ]:
print ("question1에서 최소 단어 수 : " , min(df['q1_n_words']))
print ("question2에서 최소 단어 수 : " , min(df['q2_n_words']))
print ("question1에서 최소 단어 수를 갖는 질문의 수 :", df[df['q1_n_words']== 1].shape[0])
print ("question2에서 최소 단어 수를 갖는 질문의 수 :", df[df['q2_n_words']== 1].shape[0])

In [ ]:
df[0:].head()

#### 3.3.1.1 word_share(유사도) 특성의 분석
- is_duplicate가 0이면, word_sahre가 0.1-0.2에 몰려있고, is_duplicate가 1인 경우에는 0.2-0.3 구간에 밀집되어 있는 것을 알 수 있습니다.

In [ ]:
plt.figure(figsize=(16, 8))

plt.subplot(1,2,1)
sns.violinplot(x = 'is_duplicate', y = 'word_share', data = df[0:])


plt.subplot(1,2,2)
sns.histplot(df[df['is_duplicate'] == 1.0]['word_share'][0:] , label = "1", color = '#f49093')
sns.histplot(df[df['is_duplicate'] == 0.0]['word_share'][0:] , label = "0" , color = '#9999ff')
plt.show()

#### 3.3.1.2 word_Common 특성
- is_duplicate가 0이면, 0-4에 몰려있고, is_duplicate가 1인 경우에는 4-8 구간에 밀집되어 있는 것을 알 수 있음.
- 특성상 word_share와 비슷한 추세를 나타내고 있음.

In [ ]:
plt.figure(figsize=(16, 8))

plt.subplot(1,2,1)
sns.violinplot(x = 'is_duplicate', y = 'word_Common', data = df[0:])

plt.subplot(1,2,2)
sns.histplot(df[df['is_duplicate'] == 1.0]['word_Common'][0:], kde="True", label = "1", color = '#f49093')
sns.histplot(df[df['is_duplicate'] == 0.0]['word_Common'][0:], kde="True", label = "0", color = '#9999ff')
plt.show()

## 3.4 전처리
    - html 태그 제거
    - 구두점(Punctuations) 제거
    - 형태소 분석
    - 불용어(Stopwords) 제거
    - 확장 및 수축등

In [ ]:
# english 불용어(stopwords) 리스트 
from nltk.corpus import stopwords
# To get the results in 4 decimal points
SAFE_DIV = 0.0001 
STOP_WORDS = stopwords.words("english")
print(STOP_WORDS, len(STOP_WORDS))
# 179개의 불용어(STOP_WORD)가 있음을 알 수 있다.

In [ ]:
# 전각문자, 기호문자, 줄임 문자 등을 명확한 단어로 대체
def preprocess(x):
    print("== START preprocess x: %s" % x)
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
       .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
       .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
       .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
       .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
       .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
       .replace("€", " euro ").replace("'ll", " will")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    porter = PorterStemmer()
    pattern = re.compile('\W')
    
    if type(x) == type(''):
        x = re.sub(pattern, ' ', x)
    
    if type(x) == type(''):
        x = porter.stem(x)
        example1 = BeautifulSoup(x)
        x = example1.get_text()
            
    print("== END preprocess x: %s" % x)
    return x

## 3.5 고급 특성 추출 (NLP와 Fuzzy 특성)

정의:
- __Token__: 공백 문자로 문장을 분할하여 Token을 얻을 수 있습니다.
- __Stop_Word__ : NLTK에 따른 stopword.
- __Word__ : stopword가 아닌 token


Features:
- __cwc_min__ : Q1과 Q2의 최단 길이 단어 수에 대한 common_word_count의 비율 <br>cwc_min = common_word_count / (min(len(q1_words), len(q2_words))
<br>
- __cwc_max__ : Q1과 Q2의 최장 길이 단어 수에 대한 common_word_count의 비율 <br>cwc_max = common_word_count / (max(len(q1_words), len(q2_words))
<br>
- __csc_min__ : Q1과 Q2의 최단 길이 stop ward 수에 대한 common_stop_count의 비율 <br> csc_min = common_stop_count / (min(len(q1_stops), len(q2_stops))
<br>
- __csc_max__ : Q1과 Q2의 최장 길이 stop ward 수에 대한 common_stop_count의 비율 <br>csc_max = common_stop_count / (max(len(q1_stops), len(q2_stops))
<br>
- __ctc_min__ : Q1과 Q2의 최단 길이 token 수에 대한  common_token_count의 비율 <br>ctc_min = common_token_count / (min(len(q1_tokens), len(q2_tokens))
<br>
- __ctc_max__ : Q1과 Q2의 최장 길이 token 수에 대한  common_token_count의 비율 <br>ctc_max = common_token_count / (max(len(q1_tokens), len(q2_tokens))
<br>
- __last_word_eq__ : 두 질문의 마지막 단어가 같은지 확인 <br>last_word_eq = int(q1_tokens[-1] == q2_tokens[-1])
<br>
- __first_word_eq__ : 두 질문의 첫번째 단어가 같은지 확인 <br>first_word_eq = int(q1_tokens[0] == q2_tokens[0])
<br>
- __abs_len_diff__ : 차이의 절대값 <br>abs_len_diff = abs(len(q1_tokens) - len(q2_tokens))
<br>
- __mean_len__ :  두 질문의 평균 Token 길이 <br>mean_len = (len(q1_tokens) + len(q2_tokens))/2
<br>
- __fuzz_ratio__ :  https://github.com/seatgeek/fuzzywuzzy#usage<br>
http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/
<br>
- __fuzz_partial_ratio__ :  https://github.com/seatgeek/fuzzywuzzy#usage<br>
http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/
<br>
- __token_sort_ratio__ : https://github.com/seatgeek/fuzzywuzzy#usage<br>
http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/
<br>
- __token_set_ratio__ : https://github.com/seatgeek/fuzzywuzzy#usage<br>
http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/
<br>
- __longest_substr_ratio__ : Q1 및 Q2의 토큰 개수의 최소 길이에 대한 가장 긴 공통 부분 문자열 길이의 비율 <br>longest_substr_ratio = len(longest common substring) / (min(len(q1_tokens), len(q2_tokens))

In [ ]:
def get_token_features(q1, q2):
    print("== START get_token_features q1: %s || q2: %s" % (q1, q2))
    token_features = [0.0]*10
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()

    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features
    
    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    # Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    token_features[8] = abs(len(q1_tokens) - len(q2_tokens))
    
    # Average Token Length of both Questions
    token_features[9] = (len(q1_tokens) + len(q2_tokens))/2
    return token_features

# Get the Longest Common sub string
def get_longest_substr_ratio(a, b):
    print("== START get_longest_substr_ratio a: %s, b: %s" % (a, b))
    strs = list(distance.lcsubstrings(a, b))
    if len(strs) == 0:
        return 0
    else:
        return len(strs[0]) / (min(len(a), len(b)) + 1)

def extract_features(df):
    print("== START extract_features df ==")
    print(df)
    # Pre-processing each question
    df["question1"] = df["question1"].fillna("").apply(preprocess)
    df["question2"] = df["question2"].fillna("").apply(preprocess)

    print("token features...")
    
    # Merging Features with dataset
    token_features = df.apply(lambda x: get_token_features(x["question1"], x["question2"]), axis=1)
    
    df["cwc_min"]       = list(map(lambda x: x[0], token_features))
    df["cwc_max"]       = list(map(lambda x: x[1], token_features))
    df["csc_min"]       = list(map(lambda x: x[2], token_features))
    df["csc_max"]       = list(map(lambda x: x[3], token_features))
    df["ctc_min"]       = list(map(lambda x: x[4], token_features))
    df["ctc_max"]       = list(map(lambda x: x[5], token_features))
    df["last_word_eq"]  = list(map(lambda x: x[6], token_features))
    df["first_word_eq"] = list(map(lambda x: x[7], token_features))
    df["abs_len_diff"]  = list(map(lambda x: x[8], token_features))
    df["mean_len"]      = list(map(lambda x: x[9], token_features))
   
    # Computing Fuzzy Features and Merging with Dataset
    # Do read this blog: http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/
    # https://stackoverflow.com/questions/31806695/when-to-use-which-fuzz-function-to-compare-2-strings
    # https://github.com/seatgeek/fuzzywuzzy
    print("fuzzy features..")

    df["token_set_ratio"]       = df.apply(lambda x: fuzz.token_set_ratio(x["question1"], x["question2"]), axis=1)
    # The token sort approach involves tokenizing the string in question, sorting the tokens alphabetically, and 
    # then joining them back into a string We then compare the transformed strings with a simple ratio().
    df["token_sort_ratio"]      = df.apply(lambda x: fuzz.token_sort_ratio(x["question1"], x["question2"]), axis=1)
    df["fuzz_ratio"]            = df.apply(lambda x: fuzz.QRatio(x["question1"], x["question2"]), axis=1)
    df["fuzz_partial_ratio"]    = df.apply(lambda x: fuzz.partial_ratio(x["question1"], x["question2"]), axis=1)
    df["longest_substr_ratio"]  = df.apply(lambda x: get_longest_substr_ratio(x["question1"], x["question2"]), axis=1)
    return df

In [ ]:
if os.path.isfile('../input/d/elemento/quora-question-pairs/nlp_features_train.csv'):
    print("os.path.isfile: True ")
    df = pd.read_csv("../input/d/elemento/quora-question-pairs/nlp_features_train.csv", encoding='latin-1')
    df.fillna('')
else:
    print("os.path.isfile: False")
    print("Extracting features for train:")
    df = pd.read_csv("train.csv")
    df = extract_features(df)
    df.to_csv("nlp_features_train.csv", index=False)

df.head(5)

### 3.5.1 추출된 특성의 분석
#### 3.5.1.1 Word clouds 도식화하기
- 중복 및 비중복 질문 쌍의 단어 구름 만들기
- 중복 질문 쌍과 비중복 질문 쌍에서 가장 자주 등장하는 단어를 관찰할 수 있다.
- 아래의 단어 구름을 기반으로, 자주 발생하는 중복 질문 쌍, 중복되지 않은 질문 쌍 및 모든 질문 쌍에서 자주 나타나는 특정 단어가 있음을 알 수 있다.
- 이러한 관찰은 단어의 빈도를 계산하는 것이 우리에게 중요한 기능을 제공할 수 있다.

In [ ]:
df_duplicate = df[df['is_duplicate'] == 1]
dfp_nonduplicate = df[df['is_duplicate'] == 0]

# Converting 2d array of q1 and q2 and flatten the array: like {{1,2},{3,4}} to {1,2,3,4}
p = np.dstack([df_duplicate["question1"], df_duplicate["question2"]]).flatten()
n = np.dstack([dfp_nonduplicate["question1"], dfp_nonduplicate["question2"]]).flatten()

print ("== 클래스 1에 나타난 데이터 포인터의 수 (duplicate pairs) :",len(p))
print ("== 클래스 0에 나타난 데이터 포인터의 수 (non duplicate pairs) :",len(n))

# Saving the np array into a text file
np.savetxt('train_p.txt', p, delimiter=' ', fmt='%s')
np.savetxt('train_n.txt', n, delimiter=' ', fmt='%s')

In [ ]:
# 텍스트 파일을 읽어들이고, stop word 제거
d = path.dirname('.')

textp_w = open(path.join(d, 'train_p.txt')).read()
textn_w = open(path.join(d, 'train_n.txt')).read()
stopwords = set(STOPWORDS)
stopwords.add("said")
stopwords.add("br")
stopwords.add(" ")
stopwords.remove("not")

stopwords.remove("no")
# stopwords.remove("good")
# stopwords.remove("love")
stopwords.remove("like")
# stopwords.remove("best")
# stopwords.remove("!")
print ("Total number of words in duplicate pair questions :",len(textp_w))
print ("Total number of words in non duplicate pair questions :",len(textn_w))

In [ ]:
# Word Clouds 생성
plt.figure(figsize = (12, 10))

wc = WordCloud(background_color="white", max_words=len(textp_w), stopwords=stopwords)
wc.generate(textp_w)
print ("중복 질문 쌍에 대한 Word Cloud")
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# Word Clouds generated from non duplicate pair question's text
plt.figure(figsize = (12, 10))
wc = WordCloud(background_color="white", max_words=len(textn_w),stopwords=stopwords)
wc.generate(textn_w)
print ("중복되지 않는 질문 쌍에 대한 Word Cloud")
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

#### 3.5.1.2 ['ctc_min', 'cwc_min', 'csc_min', 'token_sort_ratio'] 특성 도식화하기

In [ ]:
n = df.shape[0]
sns.pairplot(df[['ctc_min', 'cwc_min', 'csc_min', 'token_sort_ratio', 'is_duplicate']][0:n], hue='is_duplicate', vars=['ctc_min', 'cwc_min', 'csc_min', 'token_sort_ratio'])
plt.show()

In [ ]:
# token_sort_ratio의 분포
plt.figure(figsize=(16, 8))

plt.subplot(1,2,1)
sns.violinplot(x = 'is_duplicate', y = 'token_sort_ratio', data = df[0:] , )

plt.subplot(1,2,2)
sns.distplot(df[df['is_duplicate'] == 1.0]['token_sort_ratio'][0:] , label = "1", color = 'red')
sns.distplot(df[df['is_duplicate'] == 0.0]['token_sort_ratio'][0:] , label = "0" , color = 'blue' )
plt.show()

In [ ]:
# fuzz_ratio의 분포
plt.figure(figsize=(16, 8))

plt.subplot(1,2,1)
sns.violinplot(x = 'is_duplicate', y = 'fuzz_ratio', data = df[0:] , )

plt.subplot(1,2,2)
sns.distplot(df[df['is_duplicate'] == 1.0]['fuzz_ratio'][0:] , label = "1", color = 'red')
sns.distplot(df[df['is_duplicate'] == 0.0]['fuzz_ratio'][0:] , label = "0" , color = 'blue' )
plt.show()

### 3.5.2 시각화하기

In [ ]:
%%time
# 15가지 기능에 대한 차원 축소를 위해 TSNE 사용
# 3차원으로 정리
from sklearn.preprocessing import MinMaxScaler

dfp_subsampled = df[0:5000]
X = MinMaxScaler().fit_transform(dfp_subsampled[[
    'cwc_min', 'cwc_max', 'csc_min', 'csc_max', 'ctc_min', 'ctc_max', 'last_word_eq', 
    'first_word_eq', 'abs_len_diff', 'mean_len', 'token_set_ratio', 'token_sort_ratio',  
    'fuzz_ratio', 'fuzz_partial_ratio', 'longest_substr_ratio']
])
y = dfp_subsampled['is_duplicate'].values

In [ ]:
%%time
tsne2d = TSNE(
    n_components=2,
    init='random', # pca
    random_state=101,
    method='barnes_hut',
    n_iter=1000,
    verbose=2,
    angle=0.5
).fit_transform(X)

In [ ]:
%%time
df = pd.DataFrame({'x':tsne2d[:,0], 'y':tsne2d[:,1] ,'label':y})

sns.lmplot(data=df, x='x', y='y', hue='label', fit_reg=False, size=8,
   palette="Set1",markers=['s','o'])
plt.title("perplexity : {} and max_iter : {}".format(30, 1000))
plt.show()

In [ ]:
%%time
from sklearn.manifold import TSNE
tsne3d = TSNE(
    n_components=3,
    init='random', # pca
    random_state=101,
    method='barnes_hut',
    n_iter=1000,
    verbose=2,
    angle=0.5
).fit_transform(X)

In [ ]:
%%time
trace1 = go.Scatter3d(
    x=tsne3d[:,0],
    y=tsne3d[:,1],
    z=tsne3d[:,2],
    mode='markers',
    marker=dict(
        sizemode='diameter',
        color = y,
        colorscale = 'Portland',
        colorbar = dict(title = 'duplicate'),
        line=dict(color='rgb(255, 255, 255)'),
        opacity=0.75
    )
)

data=[trace1]
layout=dict(height=800, width=800, title='3d embedding with engineered features')
fig=dict(data=data, layout=layout)
py.iplot(fig, filename='3DBubble')

## 3.6 tfidf 가중 word-vector로 텍스트 데이터 특징화하기
- 앞서 분석에서 단어에 따라'클래스 0' 또는 클래스 1에서 더 자주 발생한다는 것을 알 수 있었음.
- 그래서 이를 어떻게 활용하기 위해 **GLoVE**를 사용할 수 있다.
- GLoVE는 단어 간의 **semantic relationships**를 유지하면서 단어를 벡터로 변환하는 데 도움이 된다는 점에서 W2V와 매우 유사합니다. 유일한 차이점은 작동 방식에 있습니다.

In [ ]:
# 디코딩 문제 방지하기
df = pd.read_csv("../input/quora-question-pairs/train.csv.zip")
df.head()

In [ ]:
# 질문을 unicode로 인코딩하기
# https://stackoverflow.com/a/6812069
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))
df.head()

In [ ]:
%%time
# 텍스트 병합
questions = list(df['question1']) + list(df['question2'])
print(type(questions))

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

- TF-IDF 점수를 찾고, 이 점수에 따라 각 질문을 word2vec 벡터의 가중 평균으로 변환합니다.
- 여기서 "Spacy"와 함께 사전 훈련된 GLOVE 모델을 사용합니다. https://spacy.io/usage/vectors-similarity
- 위키피디아에서 학습한 모델로 성능이 강력함.

In [ ]:
%%time

print("== spacy.load ==")
# en_vectors_web_lg에는 100만 개 이상의 고유 벡터가 포함되어 있습니다.
nlp = spacy.load('en_core_web_sm')
vecs1 = []

print("== for qu1 in tqdm(list(df['question1'])): ==")
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(df['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])    
    for word1 in doc1:
        # Word2Vec
        vec1 = word1.vector
        # Fetch df score
        try: idf = word2tfidf[str(word1)]
        except: idf = 0
        # Compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df['q1_feats_m'] = list(vecs1)

In [ ]:
%%time
vecs2 = []
for qu2 in tqdm(list(df['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc1), len(doc2[0].vector)])
    for word2 in doc2:
        # Word2Vec
        vec2 = word2.vector
        # Fetch df score
        try: idf = word2tfidf[str(word2)]
        except: idf = 0
        # Compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

In [ ]:
# prepro_features_train.csv(간단한 전처리 기능)
if os.path.isfile('../input/d/elemento/quora-question-pairs/nlp_features_train.csv'):
    print("os.path.isfile True - nlp_features_train.csv")
    dfnlp = pd.read_csv("../input/d/elemento/quora-question-pairs/nlp_features_train.csv",encoding='latin-1')
else:
    print("os.path.isfile False - nlp_features_train.csv")
    print("Download nlp_features_train.csv from drive or run previous notebook")

# nlp_features_train.csv(NLP 기능)    
if os.path.isfile('../input/d/elemento/quora-question-pairs/df_fe_without_preprocessing_train.csv'):
    print("os.path.isfile True - df_fe_without_preprocessing_train.csv")
    dfppro = pd.read_csv("../input/d/elemento/quora-question-pairs/df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("os.path.isfile False - df_fe_without_preprocessing_train.csv")
    print("Download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

- 최종 특징 벡터 또는 x_i 4개의 부분으로 구성됩니다. 첫 번째 부분은 원본 데이터 세트에서 제공한 기능과 **기본 기능 추출 섹션(3.3)**에서 엔지니어링한 기능으로 구성됩니다.
- 2부는 **Advanced Feature Extraction Section(3.5)**에서 엔지니어링한 15개의 기능으로 구성됩니다.

In [ ]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [ ]:
print("nlp 특성의 데이터 프레임")
print("nlp 데이터 프레임에 존재하는  특성의 수 :", df1.shape[1])
df1.head()

In [ ]:
print("전처리 전의 데이터")
print("전처리 전의 데이터 프레임에 존재하는 특성의 수 :", df2.shape[1])
df2.head()

In [ ]:
print("Questions 1의 tfidf-weighted word2vec")
print("question1 w2v 데이터 프레임에 존재하는 특성의 수 :", df3_q1.shape[1])
df3_q1.head()

In [ ]:
print("Questions 2의 tfidf-weighted word2vec")
print("question2 w2v 데이터 프레임에 존재하는 특성의 수 :", df3_q2.shape[1])
df3_q2.head()

In [ ]:
print("최종 데이터 프레임에 존재하는 특성의 수 :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

In [ ]:
# 최종 특성을 csv파일에 저장하기
if not os.path.isfile('../input/d/elemento/quora-question-pairs/final_features_sm.csv'):
    print("final_features_sm is not file")
    df3_q1['id'] = df1['id']
    df3_q2['id'] = df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('final_features_sm.csv')

# 4. 머신 러닝 모델
## 4.1 데이터 읽어와서 sql 테이블에 저장하기

In [ ]:
data = pd.read_csv("../input/d/elemento/quora-question-pairs/final_features_sm.csv", nrows = 100000)
print(data.shape)

y_true = data['is_duplicate']
data.drop(['Unnamed: 0', 'id', 'is_duplicate'], axis=1, inplace=True)
print(data.shape)

In [ ]:
data.head(10)

## 4.2 Random train test split(7:3)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, y_true, stratify = y_true, test_size = 0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
print("train data의 수:",X_train.shape)
print("test data의 수:",X_test.shape)

In [ ]:
print("---------- train data내의 출력 변수의 분산 ----------")
train_distr = Counter(y_train)
train_len = len(y_train)
print("Class 0: %2.2f, Class 1: %2.2f" % (int(train_distr[0])/train_len, int(train_distr[1])/train_len))

print("---------- test data 내의 출력 변수의 분산 ----------")
test_distr = Counter(y_test)
test_len = len(y_test)
print("Class 0: %2.2f, Class 1: %2.2f" % (int(test_distr[0])/test_len, int(test_distr[1])/test_len))

In [ ]:
# plot_confusion_matrix는 y_i, y_i_hat이 주어지면 confusion matrix을 표시합니다.
def plot_confusion_matrix(test_y, predict_y):
    print("== START plot_confusion_matrix ==")
    """    
    이 함수는 Seaborn 히트맵을 사용하여 3개의 다른 행렬을 플로팅합니다. 
    Confusion Matrix, Precision Matrix 및 Recall Matrix를 구성합니다.
    confusion matrix는 평소와 같으며, precision matrix에서 각 열의 값 합계는 1입니다.
    마찬가지로 Recall Matrix에서 각 행의 값의 합은 1입니다.
    """
    
    C = confusion_matrix(test_y, predict_y)
    # 열 내부의 요소의 합으로 confusion matrix의 각 요소들을 나눕니다.
    A = (((C.T)/(C.sum(axis=1))).T)
    # 행 내부의 요소의 합으로 confusion matrix의 각 요소들을 나눕니다.    
    B = (C/C.sum(axis=0))

    plt.figure(figsize=(12,10))
    
    labels = [0,1]
    cmap=sns.light_palette("blue")
    
    # heatmap 형식으로 C 표현하기
    plt.subplot(2, 2, 1)
    sns.heatmap(C, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Confusion matrix")
    
    # heatmap 형식으로 B 표현하기
    plt.subplot(2, 2, 2)
    sns.heatmap(B, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Precision matrix")
    
    # heatmap 형식으로 A 표현하기
    plt.subplot(2, 2, 3)
    sns.heatmap(A, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Recall matrix")
    
    plt.show()

## 4.4 random model 생성하기 (최악의 경우의 log-loss 찾기)

In [ ]:
# We need to generate 9 numbers and the sum of numbers should be 1
# one solution is to genarate 9 numbers and divide each of the numbers by their sum
# ref: https://stackoverflow.com/a/18662466/4084039
# We create an output array that has exactly same size as the CV data
predicted_y = np.zeros((test_len, 2))
for i in range(test_len):
    rand_probs = np.random.rand(1,2)
    predicted_y[i] = ((rand_probs / sum(sum(rand_probs)))[0])
print("Log loss on Test Data using Random Model", log_loss(y_test, predicted_y, eps=1e-15))

predicted_y = np.argmax(predicted_y, axis=1)
plot_confusion_matrix(y_test, predicted_y)

## 4.5 Logistic Regression with hyperparameter tuning# # Hyper-parameters for SGD classifier

In [ ]:

# alpha = [10 ** x for x in range(-5, 2)] 

# # Read more about SGDClassifier() at
# # http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html

# log_error_array=[]
# for i in alpha:
#     clf = SGDClassifier(alpha=i, penalty='l2', loss='log', random_state=42)
#     clf.fit(X_train, y_train)
#     sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
#     sig_clf.fit(X_train, y_train)
#     predict_y = sig_clf.predict_proba(X_test)
#     log_error_array.append(log_loss(y_test, predict_y, labels = clf.classes_))
#     print('Alpha:', i, "Log loss:", log_loss(y_test, predict_y, labels = clf.classes_))

# fig, ax = plt.subplots()
# ax.plot(alpha, log_error_array,c='g')
# for i, txt in enumerate(np.round(log_error_array, 3)):
#     ax.annotate((alpha[i], np.round(txt,3)), (alpha[i], log_error_array[i]))
# plt.grid()
# plt.title("Cross Validation Error for each alpha")
# plt.xlabel("Alpha(s)")
# plt.ylabel("Error measure")
# plt.show()

# best_alpha = np.argmin(log_error_array)
# clf = SGDClassifier(alpha=alpha[best_alpha], penalty='l2', loss='log', random_state=42)
# clf.fit(X_train, y_train)
# sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
# sig_clf.fit(X_train, y_train)

In [ ]:
# Saving the Model
# dump(sig_clf, 'logistic_regression.joblib')

# Loading the Model
sig_clf = load('../input/d/elemento/quora-question-pairs/logistic_regression.joblib')

In [ ]:
predict_y = sig_clf.predict_proba(X_train)
print("Best Alpha: 0.01, Train log loss:",log_loss(y_train, predict_y, labels=sig_clf.classes_))
predict_y = sig_clf.predict_proba(X_test)
print("Best Alpha: 0.01, Test log loss:",log_loss(y_test, predict_y, labels=sig_clf.classes_))
predicted_y = np.argmax(predict_y, axis=1)
print("Total number of data points:", len(predicted_y))
plot_confusion_matrix(y_test, predicted_y)

## 4.6 Linear SVM with hyperparameter tuning

In [ ]:
# # Hyper-parameters for SGD classifier
# alpha = [10 ** x for x in range(-5, 2)]

# # Read more about SGDClassifier() at
# # http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html

# log_error_array=[]
# for i in alpha:
#     clf = SGDClassifier(alpha=i, penalty='l1', loss='hinge', random_state=42)
#     clf.fit(X_train, y_train)
#     sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
#     sig_clf.fit(X_train, y_train)
#     predict_y = sig_clf.predict_proba(X_test)
#     log_error_array.append(log_loss(y_test, predict_y, labels=clf.classes_))
#     print('Alpha:', i, "Log loss:", log_loss(y_test, predict_y, labels = clf.classes_, eps=1e-15))

# fig, ax = plt.subplots()
# ax.plot(alpha, log_error_array,c='g')
# for i, txt in enumerate(np.round(log_error_array,3)):
#     ax.annotate((alpha[i],np.round(txt,3)), (alpha[i],log_error_array[i]))
# plt.grid()
# plt.title("Cross Validation Error for each alpha")
# plt.xlabel("Alpha(s)")
# plt.ylabel("Error measure")
# plt.show()

# best_alpha = np.argmin(log_error_array)
# clf = SGDClassifier(alpha=alpha[best_alpha], penalty='l1', loss='hinge', random_state=42)
# clf.fit(X_train, y_train)
# sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
# sig_clf.fit(X_train, y_train)

In [ ]:
# Saving the Model
# dump(sig_clf, 'linear_svm.joblib')

# Loading the Model
sig_clf = load('../input/d/elemento/quora-question-pairs/linear_svm.joblib')

In [ ]:
predict_y = sig_clf.predict_proba(X_train)
print("Best Alpha: 0.01, Train log loss:", log_loss(y_train, predict_y, labels=sig_clf.classes_))
predict_y = sig_clf.predict_proba(X_test)
print("Best Alpha: 0.01, Test log loss:", log_loss(y_test, predict_y, labels=sig_clf.classes_))
predicted_y = np.argmax(predict_y, axis=1)
print("Total number of data points:", len(predicted_y))
plot_confusion_matrix(y_test, predicted_y)

- 218차원 데이터 세트를 가지고 있으며, 매우 높지도 낮지도 않은 것으로 볼 수 있다.
- 이 경우 선형 모델과 GBDT와 같은 더 복잡한 모델을 모두 시도하여 선형 모델이 과소 적합하지 않은지 확인할 수 있다.

## 4.7 XGBoost

In [ ]:
# params = {}
# params['objective'] = 'binary:logistic'
# params['eval_metric'] = 'logloss'
# params['eta'] = 0.02
# params['max_depth'] = 4

d_train = xgb.DMatrix(X_train, label = y_train)
d_test = xgb.DMatrix(X_test, label = y_test)

# watchlist = [(d_train, 'train'), (d_test, 'valid')]
# bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=20, verbose_eval=10)

In [ ]:
# Saving the Model
# dump(bst, 'xg_boost.joblib')

# Loading the Model
bst = load('../input/d/elemento/quora-question-pairs/xg_boost.joblib')

In [ ]:
xgdmat = xgb.DMatrix(X_train,y_train)
predict_y = bst.predict(d_test)
print("Test log loss:", log_loss(y_test, predict_y, labels = [0, 1]))
predicted_y = np.array(predict_y > 0.5, dtype=int)
print("Total number of data points:", len(predicted_y))
plot_confusion_matrix(y_test, predicted_y)

# 5. Modeling 계속하기
- 로그 손실을 줄이기 위해 TF-IDF W2V로 벡터라이저와 함께 RandomsearchCV를 사용하여 XgBoost 모델의 하이퍼파라미터 튜닝을 수행합니다.
- TD_IDF 가중치 word2Vec 대신 간단한 TF-IDF 벡터를 사용하여 모델(로지스틱 회귀, Linear-SVM)을 시도합니다.
## 5.1 TF-IDF W2V로 XGBoost Hyper-parameter 조정하기

In [ ]:
# params = {
#     'n_estimators': [50, 100, 150],
#     'max_depth': [15, 20, None], 
#     'objective': ['binary:logistic'],
#     'eval_metric': ['logloss'],
# }

# xgbr = xgb.XGBClassifier(verbosity = 1)
# clf = RandomizedSearchCV(xgbr, params)
# clf.fit(X_train, y_train)
# print(clf.best_params_)

In [ ]:
# The best hyper-parameters are as follows:
# n_estimators = 150, max_depth = 15, eval_metric = logloss

# Training XGBClassifier with the best hyper-parameters
# xgbr = xgb.XGBClassifier(verbosity = 1, n_estimators = 150, max_depth = 15, 
#     objective = 'binary:logistic', eval_metric = 'logloss')
# xgbr.fit(X_train, y_train)

In [ ]:
# Saving the Model
# dump(xgbr, 'xgb_classifier_hpt.joblib')

# Loading the Model
xgbr = load('../input/d/elemento/quora-question-pairs/xgb_classifier_hpt.joblib')

In [ ]:
predict_y = xgbr.predict_proba(X_train)
print("n_estimators: , max_depth: , Train log loss:", log_loss(y_train, predict_y, labels=[0,1]))
predict_y = xgbr.predict_proba(X_test)
print("n_estimators: , max_depth: , Test log loss:", log_loss(y_test, predict_y, labels=[0,1]))
predicted_y = np.argmax(predict_y, axis=1)
print("Total number of data points:", len(predicted_y))
plot_confusion_matrix(y_test, predicted_y)

## 5.2 TF-IDF로 LR, Linear-SVM 시도하기

In [ ]:
# # Using TF-IDF
# df = pd.read_csv("../input/quora-question-pairs/train.csv.zip")
# df.dropna(axis = 0, subset = ['question1', 'question2'], inplace = True)
# print(df.shape)
# df.head()

In [ ]:
# # Merge the Questions
# questions = list(df['question1']) + list(df['question2'])

# tfidf = TfidfVectorizer(min_df = 5, max_features = 500)
# tfidf.fit(questions)

# q1 = tfidf.transform(df['question1'])
# q2 = tfidf.transform(df['question2'])
# print(q1.shape, q2.shape)

In [ ]:
# dfnlp = pd.read_csv("../input/d/elemento/quora-question-pairs/nlp_features_train.csv",encoding='latin-1')
# dfppro = pd.read_csv("../input/d/elemento/quora-question-pairs/df_fe_without_preprocessing_train.csv",encoding='latin-1')
# df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
# df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
# df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)

# # Creating a DataFrame for the TF-IDF feature vectors
# df3_q1 = pd.DataFrame.sparse.from_spmatrix(q1, index = df3.index)
# df3_q2 = pd.DataFrame.sparse.from_spmatrix(q2, index = df3.index)
# print(df3_q1.shape, df3_q2.shape)

In [ ]:
# # Making the Dataset with TF-IDF feature vectors
# df3_q1['id'] = df1['id']
# df3_q2['id'] = df1['id']
# df1  = df1.merge(df2, on='id', how='left')
# df2  = df3_q1.merge(df3_q2, on='id', how='left')
# result  = df1.merge(df2, on='id', how='left')
# result.to_csv('final_features_tfidf.csv')

In [ ]:
# Loading the TF-IDF based Dataset
data = pd.read_csv("../input/d/elemento/quora-question-pairs/final_features_tfidf.csv", nrows = 100000)
print(data.shape)
data.head()

In [ ]:
y_true = data['is_duplicate']
data.drop(['Unnamed: 0', 'id','is_duplicate'], axis=1, inplace=True)
print(data.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, y_true, stratify = y_true, test_size = 0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

### 5.2.1 Logistic Regression

In [ ]:
# # Using the hyper-parameters obtained from the previous hyper-parameter tuning
# clf = SGDClassifier(alpha=0.01, penalty='l2', loss='log', random_state=42)
# clf.fit(X_train, y_train)
# sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
# sig_clf.fit(X_train, y_train)

In [ ]:
# Saving the Model
# dump(sig_clf, 'logistic_regression_tfidf.joblib')

# Loading the Model
sig_clf = load('../input/d/elemento/quora-question-pairs/logistic_regression_tfidf.joblib')

In [ ]:
# Saving the Model
# dump(sig_clf, 'logistic_regression_tfidf.joblib')

# Loading the Model
sig_clf = load('../input/d/elemento/quora-question-pairs/logistic_regression_tfidf.joblib')

In [ ]:
predict_y = sig_clf.predict_proba(X_train)
print("Best Alpha: 0.01, Train log loss:", log_loss(y_train, predict_y, labels=sig_clf.classes_))
predict_y = sig_clf.predict_proba(X_test)
print("Best Alpha: 0.01, Test log loss:", log_loss(y_test, predict_y, labels=sig_clf.classes_))
predicted_y = np.argmax(predict_y, axis=1)
print("Total number of data points:", len(predicted_y))
plot_confusion_matrix(y_test, predicted_y)

### 5.2.2 Linear SVM

In [ ]:
# # Using the hyper-parameters obtained from the previous hyper-parameter tuning
# clf = SGDClassifier(alpha=0.01, penalty='l2', loss='hinge', random_state=42)
# clf.fit(X_train, y_train)
# sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
# sig_clf.fit(X_train, y_train)

In [ ]:
# Saving the Model
# dump(sig_clf, 'linear_svm_tfidf.joblib')

# Loading the Model
sig_clf = load('../input/d/elemento/quora-question-pairs/linear_svm_tfidf.joblib')

In [ ]:
predict_y = sig_clf.predict_proba(X_train)
print("Best Alpha: 0.01, Train log loss:", log_loss(y_train, predict_y, labels=sig_clf.classes_))
predict_y = sig_clf.predict_proba(X_test)
print("Best Alpha: 0.01, Test log loss:", log_loss(y_test, predict_y, labels=sig_clf.classes_))
predicted_y = np.argmax(predict_y, axis=1)
print("Total number of data points:", len(predicted_y))
plot_confusion_matrix(y_test, predicted_y)